## Carga de datos

Instalacion de MlFlow

In [ ]:
%%capture
!pip install mlflow

Importacion de librerias

In [ ]:
import mlflow
import os

Configuracion del servidor para MLFlow

In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        --host 0.0.0.0 -p 5000 &
"""
get_ipython().system_raw(command)

In [ ]:
%%capture
!pip install kagglehub[pandas-dataset]
!pip install optuna
!pip install keras
!pip install pyngrok
!pip install scikeras
!pip install --upgrade scikit-learn

Importacion de las librerias necesarias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import itertools
import kagglehub
from kagglehub import KaggleDatasetAdapter
import optuna
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder , OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

import keras
from sklearn.metrics import classification_report, accuracy_score
import mlflow
import os
from pyngrok import ngrok


from sklearn.model_selection import train_test_split

Token de Ngrok


In [ ]:
SEED = 7
token = "2zbl0dEaBIwHYbdy8mg0wAmtd7z_3LPkpSEY88zSz53BdUUbg" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Enlace entre MLFlow, Colab y NGrok

In [ ]:
public_url = ngrok.connect(addr=5000, proto="http")
mlflow.set_tracking_uri(public_url.public_url)
#get_ipython().system_raw("mlflow ui --port 5000 --backend-store-uri mlruns/ &")
print("MLflow UI disponible en:", public_url)

MLflow UI disponible en: NgrokTunnel: "https://79a22477d85b.ngrok-free.app" -> "http://localhost:5000"


Creacion del experimento en MlFlow


In [ ]:
#ngrok.connect(5000, "http")
#mlflow.set_tracking_uri("http://localhost:5000")
exp_id = mlflow.create_experiment(name="Prediction_addicted")

Carga de informacion en un dataset

In [ ]:
file_path = "Students Social Media Addiction.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "adilshamim8/social-media-addiction-vs-relationships",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("Columns:", df.columns)

/tmp/ipython-input-10-1368852353.py:4: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Columns: Index(['Student_ID', 'Age', 'Gender', 'Academic_Level', 'Country',
       'Avg_Daily_Usage_Hours', 'Most_Used_Platform',
       'Affects_Academic_Performance', 'Sleep_Hours_Per_Night',
       'Mental_Health_Score', 'Relationship_Status',
       'Conflicts_Over_Social_Media', 'Addicted_Score'],
      dtype='object')


Estructura inicial de la informacion cruda, 705 filas con 13 Columnas


In [ ]:
df.shape

(705, 13)

## Preprocesamiento

En la siguiente celda, eliminamos la primera fila( identificador  unico de fila) y borramos el Score = 2, ya que solo contiene un solo registro. Tambien dejamos afuera los duplicados


In [ ]:
df_clean = df.drop(columns=['Student_ID']).drop_duplicates()
df_clean = df_clean.query("Addicted_Score != 2")
X=df_clean.drop(columns=['Addicted_Score'])
y=df_clean['Addicted_Score']

In [ ]:
X.shape , y.shape

((703, 11), (703,))

De manera pedagogica, mostraremos y haremos la validacion cruzada con la informacion transformada por LabelEncoder y OneHotEncoder


In [ ]:
X_test , X_train , Y_test , Y_train  = train_test_split(X , y, test_size=0.7, random_state=7)
X_val , X_test , Y_val , Y_test  = train_test_split( X_test , Y_test , test_size=0.5, random_state=7)

In [ ]:
#Mapping academic levels
le = LabelEncoder()
X['Academic_Level'] = le.fit_transform(X['Academic_Level'])

In [ ]:
X['Academic_Level']

,Academic_Level
0,2
1,0
2,2
3,1
4,0
...,...
700,2
701,0
702,2
703,0


Discriminacion entre variables categoricas o numericas

In [ ]:
X_cat = X.select_dtypes(include=['object'])
X_num = X.select_dtypes(exclude=['object'])

In [ ]:
X_cat.shape , X_num.shape

((703, 5), (703, 6))

In [ ]:
oc = OneHotEncoder()
X_cat_encoded = oc.fit_transform(X_cat)
X_cat_encoded = pd.DataFrame(X_cat_encoded.toarray(), columns=oc.get_feature_names_out())
X_cat_encoded

,Gender_Female,Gender_Male,Country_Afghanistan,Country_Albania,Country_Andorra,Country_Argentina,Country_Armenia,Country_Australia,Country_Austria,Country_Azerbaijan,...,Most_Used_Platform_Twitter,Most_Used_Platform_VKontakte,Most_Used_Platform_WeChat,Most_Used_Platform_WhatsApp,Most_Used_Platform_YouTube,Affects_Academic_Performance_No,Affects_Academic_Performance_Yes,Relationship_Status_Complicated,Relationship_Status_In Relationship,Relationship_Status_Single
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
699,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
700,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
701,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


Creacion de dataframe que sera la entrada de los modelos

In [ ]:
X_final = pd.concat([X_num.reset_index(drop=True), X_cat_encoded.reset_index(drop=True)], axis=1 )

In [ ]:
X_final.shape

(703, 135)

Separacion de los diferentes dataset ya preprocesados(Solo para la busqueda de hiperparametros)


In [ ]:
x_test , x_train , y_test , y_train  = train_test_split(X_final , y.iloc, test_size=0.7, random_state=7)
x_val , x_test , y_val , y_test  = train_test_split( x_test , y_test , test_size=0.5, random_state=7)

In [ ]:
x_train.shape , y_train.shape , x_test.shape , y_test.shape , x_val.shape , y_val.shape

((493, 135), (493,), (105, 135), (105,), (105, 135), (105,))

## Bosque de decisión

Para el entrenamiento del modelo final, usaremos un pipeline que preprocesara algunas columnas y entrenaria el modelo,

In [ ]:
Encoder = ColumnTransformer(
    transformers=[
        ('num', OrdinalEncoder(), ['Academic_Level']),
        ('cat', OneHotEncoder( handle_unknown="ignore"), ['Gender', 'Country', 'Most_Used_Platform','Affects_Academic_Performance', 'Relationship_Status']),
        ('pass_income', 'passthrough', ['Age', 'Avg_Daily_Usage_Hours', 'Sleep_Hours_Per_Night','Mental_Health_Score', 'Conflicts_Over_Social_Media'])
    ])

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', Encoder),
    ('classifier', RandomForestClassifier())
])

con Optuna, buscamos los mejores hiperparametros

In [ ]:
def objective_RF(trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 200)
    max_depth = trial.suggest_int("max_depth", 5, 15)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 5)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 2)
    criterion = trial.suggest_categorical("criterion", ['gini','entropy'])
    model = RandomForestClassifier(
          n_estimators=n_estimators
          ,max_depth=max_depth
          ,min_samples_split=min_samples_split
          ,min_samples_leaf=min_samples_leaf
          ,criterion=criterion
          ,random_state=SEED
          ).fit(x_train, y_train)
    y_pred = model.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    return score

Creacion de estudio en optuna

In [ ]:
study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///hp.db",
    study_name="RandomForest",
    load_if_exists=True,
    )

study.optimize(func=objective_RF, n_trials=30, n_jobs=-1)

[I 2025-07-26 16:40:08,573] A new study created in RDB with name: RandomForest
[I 2025-07-26 16:40:09,706] Trial 1 finished with value: 0.9142857142857143 and parameters: {'n_estimators': 119, 'max_depth': 6, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy'}. Best is trial 1 with value: 0.9142857142857143.
[I 2025-07-26 16:40:10,134] Trial 0 finished with value: 0.9333333333333333 and parameters: {'n_estimators': 181, 'max_depth': 8, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}. Best is trial 0 with value: 0.9333333333333333.
[I 2025-07-26 16:40:10,169] Trial 2 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 33, 'max_depth': 7, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'gini'}. Best is trial 2 with value: 0.9428571428571428.
[I 2025-07-26 16:40:11,398] Trial 3 finished with value: 0.9619047619047619 and parameters: {'n_estimators': 106, 'max_depth': 11, 'min_samples_split': 5, 'min_samples_leaf': 1, 

El mejor Random Forest es el siguiente


In [ ]:
params = study.best_params
print(params)

{'n_estimators': 106, 'max_depth': 11, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}


Al usar el pipeline para entrenar el modelo final, debemos cambiar el nombre de los parametros de entrada

In [ ]:
prefixed_params = {f"classifier__{k}": v for k, v in params.items()}
prefixed_params

{'classifier__n_estimators': 106,
 'classifier__max_depth': 11,
 'classifier__min_samples_split': 5,
 'classifier__min_samples_leaf': 1,
 'classifier__criterion': 'gini'}

Corremos en MLFlow el mejor modelo de optuna con la ayuda del pipeline

In [ ]:
with mlflow.start_run(
        run_name="RF_Optuna", experiment_id=exp_id
        ) as run:

    #model = RandomForestClassifier(**params).fit(x_train, y_train)
    pipeline.set_params(**prefixed_params)
    model = pipeline.fit(X_train, Y_train)
    acc = accuracy_score(y_val, model.predict(X_val))
    mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name="Prediction_addicted",
        input_example = X_train.sample(1)
    )
    mlflow.log_params(prefixed_params)
    mlflow.log_metrics({
        "accuracy": acc
        })

2025/07/26 16:40:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer col

🏃 View run RF_Optuna at: https://79a22477d85b.ngrok-free.app/#/experiments/1/runs/87c0ab8a9dd34c5ea2547d7eda2c9a1e
🧪 View experiment at: https://79a22477d85b.ngrok-free.app/#/experiments/1


Cargamos el ultimo modelo y revisamos su comportamiento a traves de todos los scores registrados, obteniendo buenos resultados en general aunque la calificacion 9 y 3 no tengan muchos elementos

In [ ]:
best_model  = mlflow.sklearn.load_model(f"models:/Prediction_addicted/latest")
report = classification_report(Y_val, best_model.predict(X_val))
print(report)

              precision    recall  f1-score   support

           3       1.00      0.67      0.80         3
           4       0.94      0.94      0.94        16
           5       0.92      1.00      0.96        23
           6       1.00      0.91      0.95        11
           7       1.00      1.00      1.00        27
           8       0.95      1.00      0.97        18
           9       1.00      0.86      0.92         7

    accuracy                           0.96       105
   macro avg       0.97      0.91      0.93       105
weighted avg       0.96      0.96      0.96       105



Con GridSearchCV revisaremos otros hiperparametros


In [ ]:
params = {'n_estimators':range(50,101,10), 'criterion':['gini','entropy'] }
forest = RandomForestClassifier()
grid = GridSearchCV(forest, params, cv=5, verbose=2)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ....................criterion=gini, n_estimators=50; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=50; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=50; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=50; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=50; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=60; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=60; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=60; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=60; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=60; total time=   0.1s
[CV] END ....................criterion=gini, n_estimators=70; total time=   0.1s
[CV] END ....................criterion=gini, n_e

,estimator,RandomForestClassifier()
,param_grid,"{'criterion': ['gini', 'entropy'], 'n_estimators': range(50, 101, 10)}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,60


In [ ]:
grid.best_params_

{'criterion': 'gini', 'n_estimators': 60}

In [ ]:

print(classification_report(y_val, grid.predict(x_val)))

              precision    recall  f1-score   support

           3       1.00      0.67      0.80         3
           4       0.94      0.94      0.94        16
           5       0.92      1.00      0.96        23
           6       1.00      0.82      0.90        11
           7       0.96      1.00      0.98        27
           8       0.95      1.00      0.97        18
           9       1.00      0.86      0.92         7

    accuracy                           0.95       105
   macro avg       0.97      0.90      0.92       105
weighted avg       0.95      0.95      0.95       105



Veamos si podemos mejorarlo un poco más

In [ ]:
new_params = {'n_estimators': range(40,60,1), 'criterion': ['entropy']}
forest_2 = RandomForestClassifier()
grid_2 = GridSearchCV(forest_2, new_params, cv=5, verbose=2)
grid_2.fit(x_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END .................criterion=entropy, n_estimators=40; total time=   0.2s
[CV] END .................criterion=entropy, n_estimators=40; total time=   0.2s
[CV] END .................criterion=entropy, n_estimators=40; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=40; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=40; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=41; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=41; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=41; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=41; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=41; total time=   0.1s
[CV] END .................criterion=entropy, n_estimators=42; total time=   0.1s
[CV] END .................criterion=entropy, n_

,estimator,RandomForestClassifier()
,param_grid,"{'criterion': ['entropy'], 'n_estimators': range(40, 60)}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,56


In [ ]:
grid_2.best_params_

{'criterion': 'entropy', 'n_estimators': 56}

In [ ]:
with mlflow.start_run(
        run_name="RF_GS", experiment_id=exp_id
        ) as run:

    model = RandomForestClassifier(**grid_2.best_params_).fit(x_train, y_train)
    acc = accuracy_score(y_val, model.predict(x_val))
    mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name="Prediction_addicted_2",
        input_example = x_train.sample(1)
    )
    mlflow.log_params(grid_2.best_params_)
    mlflow.log_metrics({
        "accuracy": acc
        })

2025/07/26 16:40:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer col

🏃 View run RF_GS at: https://79a22477d85b.ngrok-free.app/#/experiments/1/runs/71a8e74b28ca445da5938c7bafa9dafc
🧪 View experiment at: https://79a22477d85b.ngrok-free.app/#/experiments/1


In [ ]:
best_model  = mlflow.sklearn.load_model(f"models:/Prediction_addicted_2/latest")
report = classification_report(y_val, best_model.predict(x_val))
print(report)

              precision    recall  f1-score   support

           3       1.00      0.67      0.80         3
           4       0.94      0.94      0.94        16
           5       0.92      1.00      0.96        23
           6       1.00      0.82      0.90        11
           7       0.96      1.00      0.98        27
           8       1.00      1.00      1.00        18
           9       1.00      1.00      1.00         7

    accuracy                           0.96       105
   macro avg       0.97      0.92      0.94       105
weighted avg       0.96      0.96      0.96       105



## Red neuronal

Finalmente, intentamos con una red neuronal sencilla

In [ ]:
y_train_encoded = pd.get_dummies(y_train, drop_first=False)
y_val_encoded = pd.get_dummies(y_val, drop_first=False)
y_test_encoded = pd.get_dummies(y_test, drop_first=False)

print("Shape of y_train after one-hot encoding:", y_train_encoded.shape)
print("Shape of y_val after one-hot encoding:", y_val_encoded.shape)
print("Shape of y_test after one-hot encoding:", y_test_encoded.shape)

Shape of y_train after one-hot encoding: (493, 7)
Shape of y_val after one-hot encoding: (105, 7)
Shape of y_test after one-hot encoding: (105, 7)


Para asegurar la compatibilidad del experimento y considerando que hay muy pocos registros de 'addicted score' 2 (tan pocos que todos quedaron en el split test), se eliminarán esos registros.

In [ ]:
def NN_model():
  NN = keras.models.Sequential()
  NN.add(keras.layers.Dense(109, activation='relu', input_shape = (109,) ))
  NN.add(keras.layers.Dense(256, activation='relu'))
  NN.add(keras.layers.Dense(7, activation='softmax'))
  #NN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['precision', 'recall'])
  return NN

In [ ]:
clf = KerasClassifier(
    model=NN_model,
    epochs=100,
    batch_size=8,
    verbose=1,
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
pipeline = Pipeline([
    ('preprocessor', Encoder),
    ('classifier', clf)
])

In [ ]:
with mlflow.start_run(
        run_name="NN_Optuna", experiment_id=exp_id
        ) as run:

    #model = RandomForestClassifier(**params).fit(x_train, y_train)
    #pipeline.set_params(**prefixed_params)
    model = pipeline.fit(X_train, y_train_encoded)
    acc = accuracy_score(y_val_encoded, model.predict(X_val))
    mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name="Prediction_addicted",
        input_example = X_train.sample(1)
    )
    mlflow.log_params(prefixed_params)
    mlflow.log_metrics({
        "accuracy": acc
        })

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.2962 - loss: 1.7703
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6363 - loss: 1.1438
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7016 - loss: 0.8350
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7583 - loss: 0.6254
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8546 - loss: 0.4561
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9271 - loss: 0.3299
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9325 - loss: 0.2892
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9276 - loss: 0.2455
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9189 - loss: 0.2595
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9585 - loss: 0.1586
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


2025/07/26 16:50:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step


Registered model 'Prediction_addicted' already exists. Creating a new version of this model...
2025/07/26 16:50:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Prediction_addicted, version 3
Created version '3' of model 'Prediction_addicted'.


🏃 View run NN_Optuna at: https://79a22477d85b.ngrok-free.app/#/experiments/1/runs/aebdff53876c454abd7d35350f080bef
🧪 View experiment at: https://79a22477d85b.ngrok-free.app/#/experiments/1


In [ ]:
best_model  = mlflow.sklearn.load_model(f"models:/Prediction_addicted/latest")

In [ ]:
accuracy_score(y_test_encoded, best_model.predict(X_test))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


0.9142857142857143

Como se puede ver, la red neuronal se desempeña bien, pero el bosque de decisión le gana por tres puntos porcentuales.

## Despliegue

In [ ]:
!mkdir src

In [ ]:
%%writefile ./src/mlcli.py
from argparse import ArgumentParser
import mlflow
import pandas
mlflow.set_tracking_uri("http://localhost:5000")

def main():
    parser = ArgumentParser(
            prog="Social Media Addiction Classifier",
            description="CLI para modelo de prediccion de adiccion a redes sociales"
            )
    parser.add_argument("-p", "--filepath", type=str, required=True, help="Ruta al archivo csv que contiene la serie de python")
    args = parser.parse_args()

    if args.filepath:
      try:
        data = pandas.read_csv(args.filepath, header=0).iloc[:,1:]
      except Exception as e:
        print(e)

    model = mlflow.pyfunc.load_model("models:/Prediction_addicted/1")
    prediction = model.predict(data)
    print(f"El puntaje predicho de adicción es: {prediction}")

if __name__ == "__main__":
    main()

Writing ./src/mlcli.py


In [ ]:
X_test.iloc[[3, 7, 10]].to_csv("test.csv")

In [ ]:
!python ./src/mlcli.py --help

usage: Social Media Addiction Classifier [-h] -p FILEPATH

CLI para modelo de prediccion de adiccion a redes sociales

options:
  -h, --help            show this help message and exit
  -p FILEPATH, --filepath FILEPATH
                        Ruta al archivo csv que contiene la serie de python


In [ ]:
!python ./src/mlcli.py -p ./test.csv

El puntaje predicho de adicción es: [8 8 5]
